In [1]:
import urllib
from bs4 import BeautifulSoup as bs
import pandas
import numpy as np
import os
import json
import io
from pprint import pprint
print('imported')

imported


In [2]:
baseurl = 'http://www.imsdb.com'

skiptitles = ['A.I.', 'Batman and Robin', 'Batman Forever', 'Batman Returns']
skiptitles += ['Contact', 'Dark Knight, The', 'Donnie Darko', 'Executive Decision']
skiptitles += ['Goodfellas', 'Harry Potter and the Chamber of Secrets']
skiptitles += ['Harry Potter and the Deathly Hallows Part 1', 'Harry Potter and the Goblet of Fire']
skiptitles += ['Harry Potter and the Half-Blood Prince', 'Harry Potter and the Sorcerer\'s Stone']
skiptitles += ['Lethal Weapon', 'Lethal Weapon 4', 'Matchstick Man']
skiptitles += ['Outbreak', 'Troy', 'Unforgiven']
#skiptitles
print('ok')

ok


In [3]:
theletters = ['0','A','B','C','D','E','F','G','H','I','J','K']
theletters += ['L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

therawscripts = []

for lett in theletters:
    url = baseurl + '/' +'alphabetical/{0}'.format(lett)
    page = urllib.request.urlopen(url).read()
    soup = bs(page, "html.parser")
    allp = soup.find_all('p')
    for thisp in allp:
        ascript = {}
        thispagelink = thisp.find('a')
        thislink = baseurl+urllib.parse.quote(thispagelink['href']).replace('%253f', '%3f')
        ascript['scriptpagelink'] = thislink
        ascript['title'] = thisp.text[0:thisp.text.find('(')].strip()
        ascript['version'] = thisp.text[thisp.text.find('(')+1:thisp.text.find(')')].strip()
        ascript['authors'] = thisp.text[thisp.text.find(')')+1:len(thisp.text)].strip()
        ascript['authors'] = ascript['authors'][ascript['authors'].find('Written by')+10:len(ascript['authors'])].strip().split(',')
        therawscripts.append(ascript)
print('Done! Processed {0:,.0f} titles.'.format(len(therawscripts)))

Done! Processed 1,171 titles.


In [4]:
cnt = 0
notfound = []
thescripts = []
for onescript in therawscripts:
    if (np.mod(cnt, 10) == 0):
        print('Processing title number {0:,.0f}...'.format(cnt))
    url = onescript['scriptpagelink']
    page = urllib.request.urlopen(url).read()
    soup = bs(page, "html.parser")
    if (onescript['title'] in skiptitles):
        print('Eeyore is SKIPPING title {0:,.0f}: {1:}'.format(cnt, onescript['title']))
        notfound.append(onescript)
    else:
        try:
            thatlink = baseurl+soup.find_all('p')[-1].find('a')['href']
            onescript['scriptlink'] = thatlink
            thescripts.append(onescript)
        except TypeError:
            print('Script not found for {0:} and Eeyore is SAD.'.format(onescript['title']))
            notfound.append(onescript)
        except IndexError:
            print('Script not found for {0:}, and no reviews, and Eeyore is SAD.'.format(onescript['title']))
            notfound.append(onescript)
#        print('Title {0:,.0f}: {1:}, script link = {2:}'.format(cnt, onescript['title'], onescript['scriptlink']))
    cnt = cnt + 1
print('Done! Processed {0:,.0f} titles.'.format(len(therawscripts)))
#pprint(notfound)

Processing title number 0...
Processing title number 10...
Processing title number 20...
Eeyore is SKIPPING title 25: A.I.
Processing title number 30...
Processing title number 40...
Processing title number 50...
Processing title number 60...
Processing title number 70...
Processing title number 80...
Processing title number 90...
Processing title number 100...
Processing title number 110...
Processing title number 120...
Processing title number 130...
Eeyore is SKIPPING title 137: Batman and Robin
Script not found for Batman Begins and Eeyore is SAD.
Eeyore is SKIPPING title 139: Batman Forever
Processing title number 140...
Eeyore is SKIPPING title 140: Batman Returns
Processing title number 150...
Processing title number 160...
Processing title number 170...
Processing title number 180...
Processing title number 190...
Processing title number 200...
Processing title number 210...
Processing title number 220...
Script not found for Casablanca and Eeyore is SAD.
Processing title numbe

In [12]:
#for amovie in notfound:
#    print(amovie['title'], ': ', amovie['scriptpagelink'], '(', amovie['version'], amovie['authors'],')\n')

print('OK, verified that all {0:,.0f} movies in notfound truly do not have scripts on imsdb.com.'.format(len(notfound)))

OK, verified that all 27 movies in notfound truly do not have scripts on imsdb.com.


In [13]:
with io.open('scripts_notfound.json', 'w', encoding='utf-8') as f:
    json.dump(notfound, f)
print('done')

with io.open('scripts_index.json', 'w', encoding='utf-8') as f:
    json.dump(thescripts, f)
    
print('Output files written!')

done
Output files written!
